In [111]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy.engine import reflection
from sqlalchemy import *
from sqlalchemy import Column, Integer, ForeignKey, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
import pandas as pd

In [112]:
#engine = create_engine("sqlite:///../Resources/dow.sqlite")

engine = create_engine("sqlite:///../db/database.sqlite")

In [34]:
city_sql = ('SELECT city, bikes_available, time '
           #', ROW_NUMBER () OVER (PARTITION BY station_id ORDER BY COUNT(time) DESC) AS RowNum '
           'FROM status '
           'JOIN station ON status.station_id = station.id ')
           #'GROUP BY city, bikes_available ')
           #'WHERE RowNum = 1'
           #'ORDER BY bikes_available, row_ct DESC')

top_city_df2 = pd.read_sql(city_sql, engine.connect())

In [35]:
top_city_df2.head()

,city,bikes_available,time
0,San Jose,2,2013/08/29 12:06:01
1,San Jose,2,2013/08/29 12:07:01
2,San Jose,2,2013/08/29 12:08:01
3,San Jose,2,2013/08/29 12:09:01
4,San Jose,2,2013/08/29 12:10:01


In [36]:
top_city_df2['time'] = pd.to_datetime(top_city_df2['time'])

In [37]:
top_city_df2['hour'] = top_city_df2['time'].dt.hour

In [38]:
top_city_df2.head()

,city,bikes_available,time,hour
0,San Jose,2,2013-08-29 12:06:01,12
1,San Jose,2,2013-08-29 12:07:01,12
2,San Jose,2,2013-08-29 12:08:01,12
3,San Jose,2,2013-08-29 12:09:01,12
4,San Jose,2,2013-08-29 12:10:01,12


In [45]:
top_city_gb = top_city_df2.groupby(['city', 'hour'])['bikes_available'].mean().unstack(level=1)

In [44]:
top_city_gb.head()

city           hour
Mountain View  0       8.050587
               1       8.049645
               2       8.051955
               3       8.053251
               4       8.054365
Name: bikes_available, dtype: float64

In [30]:
top_sql = ('SELECT city, bikes_available, COUNT(time) AS row_ct '
           #', ROW_NUMBER () OVER (PARTITION BY station_id ORDER BY COUNT(time) DESC) AS RowNum '
           'FROM status '
           'JOIN station ON status.station_id = station.id '
           'GROUP BY city, bikes_available ')
           #'WHERE RowNum = 1'
           #'ORDER BY bikes_available, row_ct DESC')

top_city = pd.read_sql(top_sql, engine.connect())

In [32]:
top_city_df = top_city

In [ ]:
top_city_df[''] = top_city_df.groupby([''])

In [33]:
top_city_df.head(5)

,city,bikes_available,row_ct
0,Mountain View,0,24238
1,Mountain View,1,73510
2,Mountain View,2,159560
3,Mountain View,3,290038
4,Mountain View,4,475999


In [47]:
top_busiest = top_city_df.loc[top_city_df['bikes_available'] == 0]

In [48]:
top_busiest

,city,bikes_available,row_ct
0,Mountain View,0,24238
24,Palo Alto,0,5478
45,Redwood City,0,6378
72,San Francisco,0,454321
100,San Jose,0,38589


In [ ]:
top_city_df['top_Rank'] = avg_city.groupby(['city', 'yr'])[''].rank(ascending=False)

In [113]:
year_extract = '201'
avg_sql = (f'SELECT city, substr(start_date, INSTR(start_date, {year_extract}), 4) as yr, avg(duration) as avg_trip '
           'FROM trip '
           'JOIN station ON trip.start_station_id = station.id '
           'GROUP BY city, yr'
          )

avg_city = pd.read_sql(avg_sql, engine.connect())

In [114]:
avg_city['avg_trip'] = avg_city['avg_trip']/60

In [ ]:
##clean up



In [115]:
avg_city['avg_trip'] = avg_city['avg_trip'].round()
avg_city

,city,yr,avg_trip
0,Mountain View,2013,47.0
1,Mountain View,2014,29.0
2,Mountain View,2015,27.0
3,Palo Alto,2013,83.0
4,Palo Alto,2014,62.0
5,Palo Alto,2015,79.0
6,Redwood City,2013,39.0
7,Redwood City,2014,52.0
8,Redwood City,2015,34.0
9,San Francisco,2013,20.0


In [117]:
avg_list = avg_city.to_dict('records')
avg_list

[{'city': 'Mountain View', 'yr': '2013', 'avg_trip': 47.0},
 {'city': 'Mountain View', 'yr': '2014', 'avg_trip': 29.0},
 {'city': 'Mountain View', 'yr': '2015', 'avg_trip': 27.0},
 {'city': 'Palo Alto', 'yr': '2013', 'avg_trip': 83.0},
 {'city': 'Palo Alto', 'yr': '2014', 'avg_trip': 62.0},
 {'city': 'Palo Alto', 'yr': '2015', 'avg_trip': 79.0},
 {'city': 'Redwood City', 'yr': '2013', 'avg_trip': 39.0},
 {'city': 'Redwood City', 'yr': '2014', 'avg_trip': 52.0},
 {'city': 'Redwood City', 'yr': '2015', 'avg_trip': 34.0},
 {'city': 'San Francisco', 'yr': '2013', 'avg_trip': 20.0},
 {'city': 'San Francisco', 'yr': '2014', 'avg_trip': 18.0},
 {'city': 'San Francisco', 'yr': '2015', 'avg_trip': 15.0},
 {'city': 'San Jose', 'yr': '2013', 'avg_trip': 25.0},
 {'city': 'San Jose', 'yr': '2014', 'avg_trip': 22.0},
 {'city': 'San Jose', 'yr': '2015', 'avg_trip': 23.0}]

In [118]:
yr_2013 = []
yr_2014 = []
yr_2015 = []
for item in avg_list:
    data = {}
    for k,v in item.items():
        if(k=='city'):
            data['axis'] = v
        if(k=='avg_trip'):
            data['value'] = v
        if(k=='yr'and v=='2013'):
            data['name'] = v
            yr_2013.append(data)
        elif(k=='yr'and v=='2014'):
            data['name'] = v
            yr_2014.append(data)
        elif(k=='yr'and v=='2015'):
            data['name'] = v
            yr_2015.append(data)
            

In [119]:
all_yr = [yr_2013, yr_2014, yr_2015]

In [120]:
all_yr

[[{'axis': 'Mountain View', 'name': '2013', 'value': 47.0},
  {'axis': 'Palo Alto', 'name': '2013', 'value': 83.0},
  {'axis': 'Redwood City', 'name': '2013', 'value': 39.0},
  {'axis': 'San Francisco', 'name': '2013', 'value': 20.0},
  {'axis': 'San Jose', 'name': '2013', 'value': 25.0}],
 [{'axis': 'Mountain View', 'name': '2014', 'value': 29.0},
  {'axis': 'Palo Alto', 'name': '2014', 'value': 62.0},
  {'axis': 'Redwood City', 'name': '2014', 'value': 52.0},
  {'axis': 'San Francisco', 'name': '2014', 'value': 18.0},
  {'axis': 'San Jose', 'name': '2014', 'value': 22.0}],
 [{'axis': 'Mountain View', 'name': '2015', 'value': 27.0},
  {'axis': 'Palo Alto', 'name': '2015', 'value': 79.0},
  {'axis': 'Redwood City', 'name': '2015', 'value': 34.0},
  {'axis': 'San Francisco', 'name': '2015', 'value': 15.0},
  {'axis': 'San Jose', 'name': '2015', 'value': 23.0}]]

In [121]:
year_extract = '201'
avg_station_sql = (f'SELECT city, name, substr(start_date, INSTR(start_date, {year_extract}), 4) as yr, ' +
                'avg(duration) as avg_trip ' +
                'FROM trip ' +
                'JOIN station ON trip.start_station_id = station.id ' +
                'GROUP BY city, name, yr'
            )

avg_station = pd.read_sql(avg_station_sql, engine.connect())

In [122]:
avg_station.head()

,city,name,yr,avg_trip
0,Mountain View,Castro Street and El Camino Real,2014,1830.964374
1,Mountain View,Castro Street and El Camino Real,2015,1922.758454
2,Mountain View,Evelyn Park and Ride,2013,1004.867550
3,Mountain View,Evelyn Park and Ride,2014,1212.192265
4,Mountain View,Evelyn Park and Ride,2015,1066.955810


In [123]:
avg_station['avg_trip'] = avg_station['avg_trip']/60
avg_station['avg_trip'] = avg_station['avg_trip'].round()

In [124]:
avg_station_list = avg_station.to_dict('records')
avg_station_list

[{'city': 'Mountain View',
  'name': 'Castro Street and El Camino Real',
  'yr': '2014',
  'avg_trip': 31.0},
 {'city': 'Mountain View',
  'name': 'Castro Street and El Camino Real',
  'yr': '2015',
  'avg_trip': 32.0},
 {'city': 'Mountain View',
  'name': 'Evelyn Park and Ride',
  'yr': '2013',
  'avg_trip': 17.0},
 {'city': 'Mountain View',
  'name': 'Evelyn Park and Ride',
  'yr': '2014',
  'avg_trip': 20.0},
 {'city': 'Mountain View',
  'name': 'Evelyn Park and Ride',
  'yr': '2015',
  'avg_trip': 18.0},
 {'city': 'Mountain View',
  'name': 'Mountain View Caltrain Station',
  'yr': '2013',
  'avg_trip': 44.0},
 {'city': 'Mountain View',
  'name': 'Mountain View Caltrain Station',
  'yr': '2014',
  'avg_trip': 21.0},
 {'city': 'Mountain View',
  'name': 'Mountain View Caltrain Station',
  'yr': '2015',
  'avg_trip': 21.0},
 {'city': 'Mountain View',
  'name': 'Mountain View City Hall',
  'yr': '2013',
  'avg_trip': 20.0},
 {'city': 'Mountain View',
  'name': 'Mountain View City Hall

In [127]:
yrs_2013 = []
yrs_2014 = []
yrs_2015 = []

for item in avg_station_list:
    for k,v in item.items():
        print(k)

city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name
yr
avg_trip
city
name


In [128]:
yrs_2013 = []
yrs_2014 = []
yrs_2015 = []

for item in avg_station_list:
    data = {}
    for k,v in item.items():
        if(k=='name'):
            data['axis'] = v
        if(k=='avg_trip'):
            data['value'] = v
        if(k=='city'):
            data['city'] = v
        if(k=='yr'and v=='2013'):
            data['name'] = v
            yrs_2013.append(data)
        elif(k=='yr'and v=='2014'):
            data['name'] = v
            yrs_2014.append(data)
        elif(k=='yr'and v=='2015'):
            data['name'] = v
            yrs_2015.append(data)
    
all_yr_station = [yrs_2013, yrs_2014, yrs_2015]

In [129]:
all_yr_station

[[{'city': 'Mountain View',
   'axis': 'Evelyn Park and Ride',
   'name': '2013',
   'value': 17.0},
  {'city': 'Mountain View',
   'axis': 'Mountain View Caltrain Station',
   'name': '2013',
   'value': 44.0},
  {'city': 'Mountain View',
   'axis': 'Mountain View City Hall',
   'name': '2013',
   'value': 20.0},
  {'city': 'Mountain View',
   'axis': 'Rengstorff Avenue / California Street',
   'name': '2013',
   'value': 134.0},
  {'city': 'Mountain View',
   'axis': 'San Antonio Caltrain Station',
   'name': '2013',
   'value': 44.0},
  {'city': 'Palo Alto',
   'axis': 'California Ave Caltrain Station',
   'name': '2013',
   'value': 94.0},
  {'city': 'Palo Alto',
   'axis': 'Cowper at University',
   'name': '2013',
   'value': 49.0},
  {'city': 'Palo Alto',
   'axis': 'Palo Alto Caltrain Station',
   'name': '2013',
   'value': 50.0},
  {'city': 'Palo Alto',
   'axis': 'Park at Olive',
   'name': '2013',
   'value': 80.0},
  {'city': 'Palo Alto',
   'axis': 'University and Emerson

In [14]:
meta = MetaData()
meta.reflect(bind=engine)

In [15]:
insp = reflection.Inspector.from_engine(engine)
print(insp.get_table_names())

['station', 'status', 'trip', 'weather']


In [16]:
station = meta.tables['station']
status = meta.tables['status']
weather = meta.tables['weather']
trip = meta.tables['trip']

In [10]:
import pandas as pd
#result = engine.execute('SELECT COUNT(*) FROM status WHERE station_id = :val', {'val': 2})
result = engine.execute('SELECT * FROM station')
for c in result:
    print(c[1])

San Jose Diridon Caltrain Station
San Jose Civic Center
Santa Clara at Almaden
Adobe on Almaden
San Pedro Square
Paseo de San Antonio
San Salvador at 1st
Japantown
San Jose City Hall
MLK Library
SJSU 4th at San Carlos
St James Park
Arena Green / SAP Center
SJSU - San Salvador at 9th
Franklin at Maple
Redwood City Caltrain Station
San Mateo County Center
Redwood City Public Library
Stanford in Redwood City
Redwood City Medical Center
Mountain View City Hall
Mountain View Caltrain Station
San Antonio Caltrain Station
Evelyn Park and Ride
San Antonio Shopping Center
Castro Street and El Camino Real
Rengstorff Avenue / California Street
Palo Alto Caltrain Station
University and Emerson
California Ave Caltrain Station
Cowper at University
Park at Olive
Powell Street BART
Clay at Battery
Davis at Jackson
Commercial at Montgomery
Washington at Kearney
Post at Kearney
Embarcadero at Vallejo
Spear at Folsom
Harry Bridges Plaza (Ferry Building)
Embarcadero at Folsom
Embarcadero at Bryant
Tempora

In [12]:
top_sql = ('SELECT name, station_id, bikes_available, COUNT(time) AS row_ct '
           #', ROW_NUMBER () OVER (PARTITION BY station_id ORDER BY COUNT(time) DESC) AS RowNum '
           'FROM status '
           'JOIN station ON status.station_id = station.id '
           'GROUP BY name, station_id, bikes_available '
           #'WHERE RowNum = 1'
           'ORDER BY bikes_available, row_ct DESC')


resultDF = pd.read_sql('SELECT * FROM station', engine.connect())



In [14]:
resultDF["name"].tolist()

['San Jose Diridon Caltrain Station',
 'San Jose Civic Center',
 'Santa Clara at Almaden',
 'Adobe on Almaden',
 'San Pedro Square',
 'Paseo de San Antonio',
 'San Salvador at 1st',
 'Japantown',
 'San Jose City Hall',
 'MLK Library',
 'SJSU 4th at San Carlos',
 'St James Park',
 'Arena Green / SAP Center',
 'SJSU - San Salvador at 9th',
 'Franklin at Maple',
 'Redwood City Caltrain Station',
 'San Mateo County Center',
 'Redwood City Public Library',
 'Stanford in Redwood City',
 'Redwood City Medical Center',
 'Mountain View City Hall',
 'Mountain View Caltrain Station',
 'San Antonio Caltrain Station',
 'Evelyn Park and Ride',
 'San Antonio Shopping Center',
 'Castro Street and El Camino Real',
 'Rengstorff Avenue / California Street',
 'Palo Alto Caltrain Station',
 'University and Emerson',
 'California Ave Caltrain Station',
 'Cowper at University',
 'Park at Olive',
 'Powell Street BART',
 'Clay at Battery',
 'Davis at Jackson',
 'Commercial at Montgomery',
 'Washington at Kearn

In [9]:
top_sql = ('SELECT name, station_id, bikes_available, COUNT(time) AS row_ct '
           #', ROW_NUMBER () OVER (PARTITION BY station_id ORDER BY COUNT(time) DESC) AS RowNum '
           'FROM status '
           'JOIN station ON status.station_id = station.id '
           'GROUP BY name, station_id, bikes_available '
           #'WHERE RowNum = 1'
           'ORDER BY bikes_available, row_ct DESC')

top_city = pd.read_sql(top_sql, engine.connect())

In [5]:
top_city.head(5)

,name,station_id,bikes_available,row_ct
0,Commercial at Montgomery,45,0,29198
1,Market at 4th,76,0,24725
2,Embarcadero at Vallejo,48,0,24401
3,2nd at Folsom,62,0,24077
4,Embarcadero at Sansome,60,0,23143


In [101]:
# automap base
Base = automap_base()

# pre-declare User for the 'user' table
class status(Base):
    __tablename__ = 'status'

    # override schema elements like Columns
    id = Column('station_id', String, primary_key=True)

    # override relationships too, if desired.
    # we must use the same name that automap would use for the
    # relationship, and also must refer to the class name that automap will
    # generate for "address"
    #station_collection = relationship("station", collection_class=set)

# reflect
engine = create_engine("sqlite:///../db/database.sqlite")
Base.prepare(engine, reflect=True)

# we still have Address generated from the tablename "address",
# but User is the same as Base.classes.User now

Station = Base.classes.station

#u1 = session.query(User).first()
#print (u1.address_collection)

# the backref is still there:
#a1 = session.query(Address).first()
#print (a1.user)

In [85]:
#engine = create_engine("sqlite:///mydatabase.db")

# produce our own MetaData object
metadata = MetaData()

# we can reflect it ourselves from a database, using options
# such as 'only' to limit what tables we look at...
metadata.reflect(engine)

# ... or just define our own Table objects with it (or combine both)

# we can then produce a set of mappings from this MetaData.
Base = automap_base(metadata=metadata)

# calling prepare() just sets up mapped classes and relationships.


In [87]:
Base.classes.keys()

['station', 'trip']

In [67]:
# mapped classes are ready
User, Address, Order = Base.classes.user, Base.classes.address,\
    Base.classes.user_order



#Base = automap_base()

#Create and engine and get the metadata
Base = declarative_base()
metadata = MetaData(bind=engine)

In [ ]:
#Base.prepare(engine, reflect=True)

Base = declarative_base()
metadata = MetaData(bind=engine)

station = Table('station', metadata, autoload=True, autoload_with=engine)
status = Table('status', metadata, autoload=True, autoload_with=engine)
weather = Table('weather', metadata, autoload=True, autoload_with=engine)
trip = Table('trip', metadata, autoload=True, autoload_with=engine)

In [110]:
status_list = session.query(status)

In [ ]:
for c in status_list:
    print(c.station_id)

In [ ]:
for c in status.columns:
    print(c.name)

In [69]:
Base.classes.keys()

AttributeError: type object 'Base' has no attribute 'classes'

In [59]:
from sqlalchemy import inspect

In [60]:
inspector = inspect(engine)

for table_name in inspector.get_table_names():
    print(table_name)

station
status
trip
weather


In [61]:
status = Base.classes.status

AttributeError: status

In [20]:
from sqlalchemy import MetaData
from sqlalchemy import Table

metadata = MetaData()
print(metadata.tables)

immutabledict({})


In [21]:
metadata.reflect(bind=engine)
print(metadata.tables)

immutabledict({'station': Table('station', MetaData(bind=None), Column('id', INTEGER(), table=<station>, primary_key=True, nullable=False), Column('name', TEXT(), table=<station>), Column('lat', NUMERIC(), table=<station>), Column('long', NUMERIC(), table=<station>), Column('dock_count', INTEGER(), table=<station>), Column('city', TEXT(), table=<station>), Column('installation_date', TEXT(), table=<station>), schema=None), 'status': Table('status', MetaData(bind=None), Column('station_id', INTEGER(), table=<status>), Column('bikes_available', INTEGER(), table=<status>), Column('docks_available', INTEGER(), table=<status>), Column('time', TEXT(), table=<status>), schema=None), 'trip': Table('trip', MetaData(bind=None), Column('id', INTEGER(), table=<trip>, primary_key=True, nullable=False), Column('duration', INTEGER(), table=<trip>), Column('start_date', TEXT(), table=<trip>), Column('start_station_name', TEXT(), table=<trip>), Column('start_station_id', INTEGER(), table=<trip>), Colum

In [44]:
statusTable = Table('status', metadata, Column('station_id', Integer, primary_key=True), autoload=True, autoload_with=engine)
for c in statusTable.columns:
    print(c.name)


InvalidRequestError: Table 'status' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [26]:
stationTable = Table('station', metadata, autoload=True, autoload_with=engine)
for c in stationTable.columns:
    print(c.name)

id
name
lat
long
dock_count
city
installation_date


In [27]:
tripTable = Table('trip', metadata, autoload=True, autoload_with=engine)
for c in tripTable.columns:
    print(c.name)

id
duration
start_date
start_station_name
start_station_id
end_date
end_station_name
end_station_id
bike_id
subscription_type
zip_code


In [29]:
weatherTable = Table('weather', metadata, autoload=True, autoload_with=engine)
weatherCol = []
for c in weatherTable.columns:
    weatherCol.append(c.name)


['date']
['date', 'max_temperature_f']
['date', 'max_temperature_f', 'mean_temperature_f']
['date', 'max_temperature_f', 'mean_temperature_f', 'min_temperature_f']
['date', 'max_temperature_f', 'mean_temperature_f', 'min_temperature_f', 'max_dew_point_f']
['date', 'max_temperature_f', 'mean_temperature_f', 'min_temperature_f', 'max_dew_point_f', 'mean_dew_point_f']
['date', 'max_temperature_f', 'mean_temperature_f', 'min_temperature_f', 'max_dew_point_f', 'mean_dew_point_f', 'min_dew_point_f']
['date', 'max_temperature_f', 'mean_temperature_f', 'min_temperature_f', 'max_dew_point_f', 'mean_dew_point_f', 'min_dew_point_f', 'max_humidity']
['date', 'max_temperature_f', 'mean_temperature_f', 'min_temperature_f', 'max_dew_point_f', 'mean_dew_point_f', 'min_dew_point_f', 'max_humidity', 'mean_humidity']
['date', 'max_temperature_f', 'mean_temperature_f', 'min_temperature_f', 'max_dew_point_f', 'mean_dew_point_f', 'min_dew_point_f', 'max_humidity', 'mean_humidity', 'min_humidity']
['date', '

In [53]:
class status(Base):
    __tablename__ = 'status'
    
    station_id = Column(Integer, primary_key=True)

InvalidRequestError: Table 'status' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [37]:
class weather(Base):
    __tablename__ = 'user'

    id = Column(Integer, primary_key=True)

/Users/mochathereal/anaconda3/lib/python3.6/site-packages/sqlalchemy/ext/declarative/clsregistry.py:120: SAWarning: This declarative base already contains a class with the same class name and module name as sqlalchemy.ext.automap.station, and will be replaced in the string-lookup table.
  item.__name__
/Users/mochathereal/anaconda3/lib/python3.6/site-packages/sqlalchemy/ext/declarative/clsregistry.py:120: SAWarning: This declarative base already contains a class with the same class name and module name as sqlalchemy.ext.automap.trip, and will be replaced in the string-lookup table.
  item.__name__


ArgumentError: Mapper Mapper|weather|weather could not assemble any primary key columns for mapped table 'weather'

In [45]:
Base.classes.keys()

['station', 'trip']